In [ ]:
!pip install nltk scikit-learn pandas numpy matplotlib seaborn


In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [ ]:
url = "https://raw.githubusercontent.com/dD2405/Twitter_Sentiment_Analysis/master/train.csv"
data = pd.read_csv(url)

data.head()


In [ ]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@\w+|#", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    words = text.split()
    words = [w for w in words if w not in stop_words]
    return " ".join(words)

data['clean_tweet'] = data['tweet'].apply(clean_text)
data.head()


In [ ]:
import nltk
nltk.download('stopwords')


In [ ]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@\w+|#", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    words = text.split()
    words = [w for w in words if w not in stop_words]
    return " ".join(words)

data['clean_tweet'] = data['tweet'].apply(clean_text)
data.head()


In [ ]:
X = data['clean_tweet']
y = data['label']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train_tfidf, y_train)


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred)

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predicted Label")
plt.ylabel("Actual Label")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
def predict_sentiment(text):
    text = clean_text(text)
    vector = vectorizer.transform([text])
    result = model.predict(vector)
    return "Positive 😊" if result[0] == 1 else "Negative 😠"


In [ ]:
predict_sentiment("I really love this project, it's amazing!")


In [ ]:
clean_text("I really love this project, it's amazing!")


In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)


In [ ]:
predict_sentiment("I really love this project, it's amazing!")


In [ ]:
!pip install transformers torch


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification


In [ ]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


In [ ]:
def bert_predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

    return "Positive 😊" if predicted_class == 1 else "Negative 😠"


In [ ]:
bert_predict_sentiment("I really love this project, it's amazing!")


In [ ]:
bert_predict_sentiment("This is the worst experience ever")
bert_predict_sentiment("I hate this product")
bert_predict_sentiment("Not bad, could be better")
bert_predict_sentiment("Absolutely fantastic work!")


In [ ]:
print(bert_predict_sentiment("This is the worst experience ever"))
print(bert_predict_sentiment("I hate this product"))
print(bert_predict_sentiment("Not bad, could be better"))
print(bert_predict_sentiment("Absolutely fantastic work!"))


In [ ]:
!pip install langdetect sentencepiece


In [ ]:
from langdetect import detect
from transformers import MarianMTModel, MarianTokenizer


In [ ]:
translator_model_name = "Helsinki-NLP/opus-mt-mul-en"

translator_tokenizer = MarianTokenizer.from_pretrained(translator_model_name)
translator_model = MarianMTModel.from_pretrained(translator_model_name)


In [ ]:
def translate_to_english(text):
    inputs = translator_tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    translated = translator_model.generate(**inputs)

    english_text = translator_tokenizer.decode(translated[0], skip_special_tokens=True)
    return english_text


In [ ]:
def multilingual_sentiment(text):
    try:
        lang = detect(text)
    except:
        lang = "unknown"

    # If not English, translate
    if lang != "en":
        translated_text = translate_to_english(text)
    else:
        translated_text = text

    sentiment = bert_predict_sentiment(translated_text)

    return {
        "original_text": text,
        "detected_language": lang,
        "translated_text": translated_text,
        "sentiment": sentiment
    }


In [ ]:
multilingual_sentiment("I really love this project")


In [ ]:
multilingual_sentiment("ഈ പ്രോജക്ട് വളരെ നല്ലതാണ്")


In [ ]:
multilingual_sentiment("यह अनुभव बहुत खराब था")


In [ ]:
multilingual_sentiment("இந்த செயலி அருமையாக உள்ளது")


In [ ]:
multilingual_sentiment("இந்த சேவை மிகவும் மோசமாக உள்ளது")


In [ ]:
multilingual_sentiment("ഈ അനുഭവം തീരെ മോശമായിരുന്നു")


In [ ]:
multilingual_sentiment("ఈ యాప్ చాలా ఉపయోగకరంగా ఉంది")


In [ ]:
multilingual_sentiment("Cette application est vraiment géniale")


In [ ]:
multilingual_sentiment("Esta aplicación es terrible")


In [ ]:
multilingual_sentiment("Diese App ist sehr nützlich")


In [ ]:
multilingual_sentiment("このアプリは最悪です")


In [ ]:
multilingual_sentiment("이 앱은 정말 좋아요")
